This is the large casestudy for the damage dataset. Data that we are reading is 64Gb and device that I have is 32 GB so technically I cannot play around with this data

In [15]:
from dask.distributed import Client, progress
import tables
import numpy as np
Client()

/nix/store/02z0vx25y0x2arfvcp89pvy52a3q77wf-python3.7-distributed-2.9.1/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:40021 Dashboard: http://127.0.0.1:35185/status,Cluster Workers: 4 Cores: 8 Memory: 33.59 GB


In [6]:
import dask.array as da

In [9]:
path="/home/berkay/Projects/pymks/data/ms_copy_data.hdf5"


In [10]:
tps_hdf5_file = tables.open_file(path, mode='r')


In [11]:
ms_data=tps_hdf5_file.root.ms_data

In [13]:
ms_data
# Actually it has a chunk shape (from pytables) as well  maybe it could be an issue too

/ms_data (EArray(130000, 257651), shuffle, blosc(5)) ''
  atom := Float32Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (2, 257651)

# Data Construction
Creating the Large Dask Array From HDF file:<br/>
Target Data is n* 51* 51*51<br/>
So I will first read as NP array then will construct the dask array. Basically , we are stitching them together.

In [23]:
# I guess I dont need to do that
chunksize=1300
dask_arrays = []
for i in range(5):
#     ms_chunk=np.array(ms_data[i*chunksize:(i+1)*chunksize])
#     ms_chunk=ms_chunk[:,0:51**3]
#     ms_chunk=ms_chunk.reshape((chunksize,51,51,51))
    ms_chunk=da.from_array(np.array(ms_data[i*chunksize:(i+1)*chunksize,0:51**3]).reshape(chunksize,51,51,51),chunks=(chunksize,51,51,51))
    print("%d Chunks completed"%(i+1))
    dask_arrays.append(ms_chunk)

1 Chunks completed
2 Chunks completed
3 Chunks completed
4 Chunks completed
5 Chunks completed


In [ ]:
# ms_data.shape[0]//chunksize
x = da.concatenate(dask_arrays, axis=0)  # concatenate arrays along first axis

In [ ]:
x

In [ ]:
type(x)

Save the small chunk of the data with pytables

In [ ]:
h5file = tables.open_file("new_sample.h5", "w", driver="H5FD_CORE")
a = h5file.create_array(h5file.root, "array",ms_data[0:500,0:51**3])
h5file.close()

# Start from Here
Unpack the data with pytables

In [9]:
import dask.array as da
import numpy as np

In [10]:
import tables
import dask.array as da

h5file = tables.open_file("new_sample.h5", "r", driver="H5FD_CORE",driver_core_backing_store=0)

In [11]:
data=h5file.root.array

In [12]:
data=da.from_array(np.array(data).reshape(500,51,51,51),chunks=(100,51,51,51))


In [13]:
data

dask.array<array, shape=(500, 51, 51, 51), dtype=float32, chunksize=(100, 51, 51, 51), chunktype=numpy.ndarray>

In [14]:
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation,FlattenTransformer
from dask_ml.decomposition import PCA
from sklearn.pipeline import Pipeline

In [15]:
bl = Pipeline(steps=[
    ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
    ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)]))])

# There is an issue 

In [16]:
bl.transform(data)

dask.array<stack, shape=(500, 31, 31, 31, 1), dtype=float64, chunksize=(500, 31, 31, 31, 1), chunktype=numpy.ndarray>

In [17]:
bl.transform(data)[0]

dask.array<getitem, shape=(31, 31, 31, 1), dtype=float64, chunksize=(31, 31, 31, 1), chunktype=numpy.ndarray>

In [18]:
bl.transform(data)[0].compute()

array([[[[0.21717137],
         [0.21635721],
         [0.21636475],
         ...,
         [0.22320978],
         [0.22164175],
         [0.22084266]],

        [[0.2144801 ],
         [0.21472134],
         [0.21613859],
         ...,
         [0.22396363],
         [0.2223353 ],
         [0.22059389]],

        [[0.21501534],
         [0.21558827],
         [0.21800062],
         ...,
         [0.22435564],
         [0.22271977],
         [0.22110651]],

        ...,

        [[0.22944418],
         [0.22883356],
         [0.22844155],
         ...,
         [0.22231269],
         [0.2214759 ],
         [0.22198853]],

        [[0.22934618],
         [0.22906725],
         [0.22876571],
         ...,
         [0.22150606],
         [0.2216116 ],
         [0.22274238]],

        [[0.22914264],
         [0.22957988],
         [0.22977588],
         ...,
         [0.22284792],
         [0.22415964],
         [0.22547135]]],


       [[[0.21759353],
         [0.21660598],
         [0.21

This does not give a memory issue but for big data it crashes